# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 11:04AM,246514,15,8525528,"Mizner Bioscience, LLC",Released
1,Sep 15 2022 11:01AM,246513,15,30010423,"Alliance Pharma, Inc.",Released
2,Sep 15 2022 11:01AM,246513,15,30010736,"Alliance Pharma, Inc.",Released
3,Sep 15 2022 11:01AM,246513,15,30010734,"Alliance Pharma, Inc.",Released
4,Sep 15 2022 11:01AM,246513,15,30010726,"Alliance Pharma, Inc.",Released
5,Sep 15 2022 11:01AM,246513,15,30010741,"Alliance Pharma, Inc.",Released
6,Sep 15 2022 11:01AM,246513,15,30010737,"Alliance Pharma, Inc.",Released
7,Sep 15 2022 11:01AM,246513,15,30010727,"Alliance Pharma, Inc.",Released
8,Sep 15 2022 11:01AM,246513,15,30010733,"Alliance Pharma, Inc.",Released
9,Sep 15 2022 11:01AM,246513,15,30010740,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,246510,Released,1
41,246511,Released,18
42,246512,Released,1
43,246513,Released,46
44,246514,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246510,NaN,1.0
246511,NaN,18.0
246512,NaN,1.0
246513,NaN,46.0
246514,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246066,1.0,0.0
246067,0.0,1.0
246068,0.0,1.0
246278,0.0,1.0
246282,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246066,1,0
246067,0,1
246068,0,1
246278,0,1
246282,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246066,1,0
1,246067,0,1
2,246068,0,1
3,246278,0,1
4,246282,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246066,1,
1,246067,,1
2,246068,,1
3,246278,,1
4,246282,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC"
1,Sep 15 2022 11:01AM,246513,15,"Alliance Pharma, Inc."
47,Sep 15 2022 10:52AM,246512,10,Yusen – 3D Matrix
48,Sep 15 2022 10:51AM,246511,10,Beach Products Inc
66,Sep 15 2022 10:44AM,246510,10,TherapeuticsMD Elli Sample
67,Sep 15 2022 10:43AM,246508,16,"SHL Pharma, LLC"
68,Sep 15 2022 10:42AM,246507,10,MedStone Pharma LLC
79,Sep 15 2022 10:41AM,246505,10,Eywa Pharma Inc.
97,Sep 15 2022 10:38AM,246506,19,"SD Head USA, LLC"
98,Sep 15 2022 10:35AM,246504,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC",,1
1,Sep 15 2022 11:01AM,246513,15,"Alliance Pharma, Inc.",,46
2,Sep 15 2022 10:52AM,246512,10,Yusen – 3D Matrix,,1
3,Sep 15 2022 10:51AM,246511,10,Beach Products Inc,,18
4,Sep 15 2022 10:44AM,246510,10,TherapeuticsMD Elli Sample,,1
5,Sep 15 2022 10:43AM,246508,16,"SHL Pharma, LLC",,1
6,Sep 15 2022 10:42AM,246507,10,MedStone Pharma LLC,,11
7,Sep 15 2022 10:41AM,246505,10,Eywa Pharma Inc.,2,16
8,Sep 15 2022 10:38AM,246506,19,"SD Head USA, LLC",,1
9,Sep 15 2022 10:35AM,246504,10,Emerginnova,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC",1,
1,Sep 15 2022 11:01AM,246513,15,"Alliance Pharma, Inc.",46,
2,Sep 15 2022 10:52AM,246512,10,Yusen – 3D Matrix,1,
3,Sep 15 2022 10:51AM,246511,10,Beach Products Inc,18,
4,Sep 15 2022 10:44AM,246510,10,TherapeuticsMD Elli Sample,1,
5,Sep 15 2022 10:43AM,246508,16,"SHL Pharma, LLC",1,
6,Sep 15 2022 10:42AM,246507,10,MedStone Pharma LLC,11,
7,Sep 15 2022 10:41AM,246505,10,Eywa Pharma Inc.,16,2
8,Sep 15 2022 10:38AM,246506,19,"SD Head USA, LLC",1,
9,Sep 15 2022 10:35AM,246504,10,Emerginnova,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC",1,
1,Sep 15 2022 11:01AM,246513,15,"Alliance Pharma, Inc.",46,
2,Sep 15 2022 10:52AM,246512,10,Yusen – 3D Matrix,1,
3,Sep 15 2022 10:51AM,246511,10,Beach Products Inc,18,
4,Sep 15 2022 10:44AM,246510,10,TherapeuticsMD Elli Sample,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC",1.0,NaN
1,Sep 15 2022 11:01AM,246513,15,"Alliance Pharma, Inc.",46.0,NaN
2,Sep 15 2022 10:52AM,246512,10,Yusen – 3D Matrix,1.0,NaN
3,Sep 15 2022 10:51AM,246511,10,Beach Products Inc,18.0,NaN
4,Sep 15 2022 10:44AM,246510,10,TherapeuticsMD Elli Sample,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC",1.0,0.0
1,Sep 15 2022 11:01AM,246513,15,"Alliance Pharma, Inc.",46.0,0.0
2,Sep 15 2022 10:52AM,246512,10,Yusen – 3D Matrix,1.0,0.0
3,Sep 15 2022 10:51AM,246511,10,Beach Products Inc,18.0,0.0
4,Sep 15 2022 10:44AM,246510,10,TherapeuticsMD Elli Sample,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4436768,98.0,22.0
15,493027,47.0,0.0
16,985917,4.0,0.0
19,1232481,5.0,0.0
20,246497,2.0,3.0
21,2708665,7.0,4.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4436768,98.0,22.0
1,15,493027,47.0,0.0
2,16,985917,4.0,0.0
3,19,1232481,5.0,0.0
4,20,246497,2.0,3.0
5,21,2708665,7.0,4.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,98.0,22.0
1,15,47.0,0.0
2,16,4.0,0.0
3,19,5.0,0.0
4,20,2.0,3.0
5,21,7.0,4.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,98.0
1,15,Released,47.0
2,16,Released,4.0
3,19,Released,5.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Executing,22.0,0.0,0.0,0.0,3.0,4.0
Released,98.0,47.0,4.0,5.0,2.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Executing,22.0,0.0,0.0,0.0,3.0,4.0
1,Released,98.0,47.0,4.0,5.0,2.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Executing,22.0,0.0,0.0,0.0,3.0,4.0
1,Released,98.0,47.0,4.0,5.0,2.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()